In [1]:
import matplotlib.pyplot as plt 
import numpy as np

# Create data for training

In [2]:
from typing import List
import math


def create_binary_list_from_int(number: int) -> List[int]:
    if number < 0 or type(number) is not int:
        raise ValueError("Only Positive integers are allowed")

    return [int(x) for x in list(bin(number))[2:]]


max_value = 200
n_samples = 50000

input_length = int(math.log(max_value, 2)) + 1
sampled_integers = np.random.randint(0, int(max_value / 2), n_samples)

# Generate a list of binary numbers for training.
data = [create_binary_list_from_int(int(x * 2)) for x in sampled_integers]

data = [([0] * (input_length - len(x))) + x for x in data]

data = np.array(data)

# Define Discriminator & Generator

## Layer's Architecture

In [2]:
import torch
import torch.nn as nn


class Generator(nn.Module):
    def __init__(self, input_length: int):
        super(Generator, self).__init__()
        self.dense_layer = nn.Linear(int(input_length), int(input_length))
        self.activation = nn.Sigmoid()

    def forward(self, x):
        return self.activation(self.dense_layer(x))


class Discriminator(nn.Module):
    def __init__(self, input_length: int):
        super(Discriminator, self).__init__()
        self.dense = nn.Linear(int(input_length), 1);
        self.activation = nn.Sigmoid()

    def forward(self, x):
        return self.activation(self.dense(x))
    

## Optimizers and loss functions

In [4]:
generator = Generator(input_length)
discriminator = Discriminator(input_length)
loss = nn.BCELoss()
generator_optimizer = torch.optim.Adam(generator.parameters(), lr=0.001)

discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr=0.001)

# Train GAN

In [5]:
from trainers.general_trainer import Trainer

trainer = Trainer(generator, discriminator, loss, loss, generator_optimizer, discriminator_optimizer)
trainer.train_general(data, input_length, 16, 1000)

In [6]:
noise = torch.randint(0, 2, size=(10, input_length)).float()
#print(noise)
fake_data = trainer.generator(noise)


data = fake_data[0]
boolean_tensor = (fake_data>0.5).to(torch.int)
valor_entero = np.packbits(boolean_tensor)
print(valor_entero)

[42  0  0  8  0 42 40 42  8  0]


In [7]:
batch_size = 16
image_height = 128
image_width = 128
num_channels = 3

# Generate random input data
random_input = torch.randn(batch_size, num_channels, image_height, image_width)

# Print the shape of the random input data
print(random_input.shape)


torch.Size([16, 3, 128, 128])


In [23]:
class SimpleDiscriminator(nn.Module):
    def __init__(self):
        super(SimpleDiscriminator, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=5, stride=2, padding=2), # 64x64x64
            nn.LeakyReLU(negative_slope=0.2),

            nn.Conv2d(64, 128, kernel_size=5, stride=2, padding=2), # 32x32x128
            nn.BatchNorm2d(num_features=128, momentum=0.3),
            nn.LeakyReLU(negative_slope=0.2),
            
            nn.Conv2d(128, 256, kernel_size=5, stride=2, padding=2), # 16x16x256
            nn.BatchNorm2d(num_features=256, momentum=0.3),
            nn.LeakyReLU(negative_slope=0.2),

            nn.Conv2d(256, 512, kernel_size=5, stride=2, padding=2), # 8x8x512
            nn.BatchNorm2d(num_features=512, momentum=0.3),
            nn.LeakyReLU(negative_slope=0.2),
            
            nn.Conv2d(512, 1024, kernel_size=5, stride=2, padding=2), # 4x4x1024
            nn.BatchNorm2d(num_features=1024, momentum=0.3),
            nn.LeakyReLU(negative_slope=0.2),
        
        )
        self.classifier = nn.Sequential(
            nn.Linear(4*4*1024, 1),            
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(-1, 4*4*1024)
        x = self.classifier(x)
        return x

model = SimpleDiscriminator()
model(random_input)


tensor([[0.5759],
        [0.3627],
        [0.3682],
        [0.4661],
        [0.4767],
        [0.2995],
        [0.4775],
        [0.4043],
        [0.4585],
        [0.4238],
        [0.4928],
        [0.5851],
        [0.4684],
        [0.3105],
        [0.4709],
        [0.4389]], grad_fn=<SigmoidBackward0>)

In [ ]:

# class SimpleGenerator(nn.Module):
#     def __init__(self):
#         super(SimpleGenerator, self).__init__()
#         self.features = nn.Sequential(
#             nn.Linear(100, 1024*4*4),
#             nn.BatchNorm1d(1024*4*4),
#             nn.LeakyReLU(negative_slope=0.2),
#             nn.resha
#         )


def build_generator():
    model = Sequential()
    model.add(layers.Dense(1024*4*4, input_shape=(100,)))
    model.add(layers.BatchNormalization(momentum=0.3))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Reshape((4,4,1024))) # 4x4x1024

    model.add(layers.Conv2DTranspose(512,(5,5), strides=(2,2), padding='same')) # 8x8x512
    model.add(layers.BatchNormalization(momentum=0.3))
    model.add(layers.LeakyReLU(alpha=0.2))
    
    model.add(layers.Conv2DTranspose(256,(5,5), strides=(2,2), padding='same')) # 16x16x256
    model.add(layers.BatchNormalization(momentum=0.3))
    model.add(layers.LeakyReLU(alpha=0.2))
    
    model.add(layers.Conv2DTranspose(128,(5,5), strides=(2,2), padding='same')) # 32x32x128
    model.add(layers.BatchNormalization(momentum=0.3))
    model.add(layers.LeakyReLU(alpha=0.2))
    
    model.add(layers.Conv2DTranspose(64,(5,5), strides=(2,2), padding='same')) # 64x64x64
    model.add(layers.BatchNormalization(momentum=0.3))
    model.add(layers.LeakyReLU(alpha=0.2))
    
    model.add(layers.Conv2DTranspose(3,(5,5), strides=(2,2), padding='same')) # 128x128x3 --> desired output
    model.add(layers.Activation("tanh"))
    adam = optimizers.Adam(learning_rate=0.001, beta_1=0.5)
    model.compile(optimizer=adam, loss="binary_crossentropy")
    return model

In [10]:
class SimpleRegressionModel(nn.Module):
    def __init__(self):
        super(SimpleRegressionModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc = nn.Sequential(
            nn.Linear(128 * 32 * 32, 256),  # Adjust input size based on your image dimensions
            nn.ReLU(inplace=True),
            nn.Linear(256, 1)  # Output layer with one value
        )

    def forward(self, x):
        x = self.features(x)
        print(x.shape)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.fc(x)
        return x

# Create an instance of the model
model = SimpleRegressionModel()


model(random_input)

torch.Size([16, 128, 32, 32])


tensor([[-0.0240],
        [-0.0417],
        [-0.0161],
        [-0.0611],
        [ 0.0253],
        [-0.0455],
        [-0.0470],
        [-0.0672],
        [-0.0669],
        [-0.0577],
        [-0.0032],
        [-0.0449],
        [-0.0021],
        [-0.0019],
        [-0.0376],
        [-0.0169]], grad_fn=<AddmmBackward0>)